In [2]:
#Question 13 and 14

#import packages
import pandas as pd
import numpy as np 
import time
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import matplotlib.pyplot as pt
from sklearn import svm

/home/sukhjinder/anaconda/lib/python2.7/site-packages/pandas/_libs/__init__.py:4: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from .tslib import iNaT, NaT, Timestamp, Timedelta, OutOfBoundsDatetime
/home/sukhjinder/anaconda/lib/python2.7/site-packages/pandas/__init__.py:26: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from pandas._libs import (hashtable as _hashtable,
/home/sukhjinder/anaconda/lib/python2.7/site-packages/pandas/core/dtypes/common.py:6: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from pandas._libs import algos, lib
/home/sukhjinder/anaconda/lib/python2.7/site-packages/pandas/core/util/hashing.py:7: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from pandas._libs import hashing, tslib
/home/sukhjinder/anaconda/lib/python2.7/site-packages/pandas/core/i

In [15]:
def read_data(train_path, dev_path, test_path):
    
    #reading the data set
    train = pd.read_csv(train_path, header=None) 
    dev = pd.read_csv(dev_path, header=None)
    test = pd.read_csv(test_path, header=None)
    
    #combining the data set
    data = pd.concat([train,dev,test], keys=[0,1,2])
    
    #Convert categorical variable into indicator variables
    encoded_data = pd.get_dummies(data,columns=[1,2,3,4,5,6,8])
    
    #split the data into train, development and test
    train_data = encoded_data.xs(0)
    dev_data = encoded_data.xs(1)
    test_data = encoded_data.xs(2)
    
    #retrieving y label from the data
    ylabel_train = train_data[[9]]
    ylabel_dev = dev_data[[9]]
    ylabel_test = test_data[[9]]
   
    
    train_y = ylabel_train.iloc[:,:].values
    dev_y = ylabel_dev.iloc[:,:].values
    test_y = ylabel_test.iloc[:,:].values
    
   
    train_y = train_y.reshape((train_y.shape[0]))
    dev_y = dev_y.reshape((dev_y.shape[0]))
    test_y = test_y.reshape((test_y.shape[0]))
    
    #convert the y label values to 1 and -1 format
    train_y[train_y == ' <=50K'] = -1
    dev_y[dev_y == ' <=50K'] = -1
    test_y[test_y == ' <=50K'] = -1
    
    train_y[train_y == ' >50K'] = 1
    dev_y[dev_y == ' >50K'] = 1
    test_y[test_y == ' >50K'] = 1
    
    #convert the values obtained from string into integer 
    train_y = train_y.astype('int')
    dev_y = dev_y.astype('int')
    test_y = test_y.astype('int')
    
    #from the x label drop y label column 
    train_data = train_data.drop(9, axis=1)
    dev_data = dev_data.drop(9, axis=1)
    test_data = test_data.drop(9, axis=1)

    train_x = train_data.iloc[:,:].values
    dev_x = dev_data.iloc[:,:].values
    test_x = test_data.iloc[:,:].values
    
    
    return (train_x, train_y, dev_x, dev_y, test_x, test_y)

In [ ]:
def train_pa(x, y,x1,y1,x2,y2,t):
    mistakes_pa=[]
    train_acc_pa=[]
    dev_acc_pa=[]
    test_acc_pa=[]
    w1 = np.zeros((x.shape[1]))
    
    for i in range (t):
        c2=0
        for j in range (x.shape[0]):
            
            tou=(1-(y[j]*np.dot(w1,x[j])))/(np.dot(x[j],x[j]))
            
            y_hat = np.sign(np.dot(w1, x[j]))
            
            if(y_hat == 0):
                y_hat = -1
            
            if(y_hat != y[j]):
                  w1 = w1 +(tou*y[j] * x[j])
                  c2 = c2+1
        acc1 = 1 - (c2/x.shape[0])
        acc2=test(x1,y1,w1)
        acc3=test(x2,y2,w1)
        mistakes_pa.append(c2)
        train_acc_pa.append(acc1)
        dev_acc_pa.append(acc2)
        test_acc_pa.append(acc3)
    return w1,mistakes_pa,train_acc_pa,dev_acc_pa,test_acc_pa

In [ ]:
def train_perc(x, y,x1,y1,x2,y2,t):
    mistakes_p=[]
    train_acc_p=[]
    dev_acc_p=[]
    test_acc_p=[]
    w = np.zeros((x.shape[1]))
    
    for i in range (t):  #perform 5 from 1 iterations
        c1=0
        for j in range (x.shape[0]):     #perform each iteration for all observations
            
            y_hat = np.sign(np.dot(w, x[j]))    #calculate y prediction
            
            if(y_hat == 0):
                y_hat = -1
            
            if(y_hat != y[j]):
                  w = w +(y[j] * x[j])
                  c1 = c1+1
        acc1 = 1 - (c1/x.shape[0])
        acc2=test(x1,y1,w)
        acc3=test(x2,y2,w)
        mistakes_p.append(c1) 
        train_acc_p.append(acc1)
        dev_acc_p.append(acc2)
        test_acc_p.append(acc3)
   
    return w,mistakes_p,train_acc_p,dev_acc_p,test_acc_p

In [18]:
#Calculate the accuacy
def test(x,y,w):
    c=0
    for j in range (x.shape[0]):  
        y_hat = np.sign(np.dot(w, x[j])) 
        if(y_hat == 0):
            y_hat = -1
            
        if(y_hat != y[j]):
            c = c+1
    accuracy= 1-(c/x.shape[0])
    return accuracy
    

In [ ]:
#deining curve 
def curve_p(x,y,x1,y1,x2,y2):
    dev_p=[]
    i=[]
    test_p=[]
    itr_p=5000
    while itr_p < 25000:
        itr_p+=5000
        x_p=x[:itr_p,:]
        y_p=y[:itr_p]
        w,b,acc1,acc2,acc3=train_perc(x_p,y_p,x1,y1,x2,y2,t=5)
        i.append(itr_p)
        dev_p.append(acc2[-1])
        test_p.append(acc3[-1])
    return i,dev_p,test_p

In [19]:
#Average Perceptron based on smart way
def avg_per(x,y,x1,y1,x2,y2,t):
    
    
    b=0
    beta=0
    w=np.zeros((x.shape[1]))
    u=np.zeros((x.shape[1]))
    c=1
    start=time.process_time()
    for i in range(t):
        mistakes=0
        for j in range(x.shape[0]):
            y_hat = np.sign((np.dot(w,x[j]))+b)
            if (y_hat==0):
                y_hat=-1
            if(y_hat != y[j]):
                w=w+np.dot(y[j],x[j])
                b=b+y[j]
                u=u+y[j]*c*x[j]
                beta=beta+y[j]*c
                mistakes+=1
        
            c=c+1
    end=time.process_time()
    w=w-u/c
    acc1_a = 1-(mistakes/x.shape[0])
    print('time taken to perform 5 iterations using smart way average perceptron in sec:',end-start)
    acc2_a=test(x1,y1,w)
    acc3_a=test(x2,y2,w)
    b=np.array(b-(beta/c))
    
    return w,b,acc1_a,acc2_a,acc3_a

In [20]:
#naive approach based on the average perceptron
def naive(x,y,x1,y1,x2,y2,t):
    
    
    w=np.zeros((x.shape[1]))
    wsum = np.zeros((x.shape[1]))
    c = 0
    mistakes = 0
    acc1=[]
    acc2=[]
    acc3=[]
    start=time.process_time()
    for i in range(t):
        mistakes = 0
        for j in range(x.shape[0]):
            y_h = np.sign(np.dot(w,x[j]))
            if(y_h == 0):
                y_h-1
                
            if(y_h!=y[j]):
                w = w + (y[j]*x[j])
                wsum = wsum + w
                c = c + 1 
                mistakes += 1
    end=time.process_time()            
    wavg=wsum/c
    acc1 = 1 - (mistakes/x.shape[0])
   
    print('time taken to perform 5 iterations of naive way average perceptron in sec ',end-start)
    acc2 = test(x1,y1,wavg)
    acc3 = test(x2,y2,wavg)
        
 
    return wavg,acc1,acc2,acc3
            

In [ ]:
def train_svmpoly(train_x, train_y, dev_x, dev_y, test_x, test_y):
    deg = [2,3]
    s=[]
    s_a=[]
    s_t_a=[]
    s_d_a=[]
    s_te_a=[]
    for d in deg:

        svm_model = svm.SVC(C=10**-1, kernel='poly',degree=d)
        svm_model.fit(train_x, train_y)
        s.append(sum(svm_model.n_support_))
        train_acc = svm_model.score(train_x,train_y)
        dev_acc = svm_model.score(dev_x,dev_y)
        test_acc = svm_model.score(test_x,test_y)
        
        print( train_acc, dev_acc, test_acc)
        s_a.append(d)
        s_t_a.append(train_acc)
        s_d_a.append(dev_acc)
        s_te_a.append(test_acc)
        
    graph(s_a,s,'polynomial degree','support','support vectors for polynomial',"poly.png")
    graph(s_a,s_t_a,'C values','accuracy','Accuracy curve for training data ', "polytrain.png")
    graph(s_a,s_d_a,'C values','accuracy','Accuracy curve for development data ', "polydev.png")
    graph(s_a,s_te_a,'C values','accuracy','Accuracy curve for test data ', "polytest.png")

In [22]:
#curve for passive aggressive algorithm
def curve_pa(x,y,x1,y1,x2,y2):
    dev_pa=[]
    i_pa=[]
    test_pa=[]
    itr_pa=5000
    while itr_pa < 25000:
        itr_pa+=5000
        x_pa=x[:itr_pa,:]
        y_pa=y[:itr_pa]
        w,b,acc1,acc2,acc3=train_pa(x_pa,y_pa,x1,y1,x2,y2,t=5)
        i_pa.append(itr_pa)
        dev_pa.append(acc2[-1])
        test_pa.append(acc3[-1])
    return i_pa,dev_pa,test_pa

In [24]:
def train_svm(train_x, train_y, dev_x, dev_y, test_x, test_y):
    sv=[]
    a=[]
    t_a=[]
    d_a=[]
    te_a=[]
    for i in c:
        
        svm_model = svm.SVC(C=i, kernel='linear')
        train_acc = svm_model.fit(train_x, train_y).score(train_x,train_y)
        sv.append(sum(svm_model.n_support_))
        dev_acc = svm_model.score(dev_x,dev_y)
        test_acc = svm_model.score(test_x,test_y)
        a.append(c)
        t_a.append(train_acc)
        d_a.append(dev_acc)
        te_a.append(test_acc)
        print( train_acc, dev_acc, test_acc)
    graph(a,sv,'c values','support','support vectors for linear ',"SVL.png")
    graph(a,t_a,'C values','accuracy','Accuracy curve for training data ', "ACC_train.png")
    graph(a,d_a,'C values','accuracy','Accuracy curve for development data ', "ACC_dev.png")
    graph(a,te_a,'C values','accuracy','Accuracy curve for test data ', "ACC_test.png")

In [25]:
def graph(a,b,a1,b1,c, filename):
    pt.plot(a,b)
    pt.xlabel(a1)
    pt.ylabel(b1)
    pt.title(c)
    pt.savefig(filename)
    pt.close()

In [27]:
if __name__ == '__main__':
    
    train_path = "data\\income-data\\income.train.txt"
    dev_path = "data\\income-data\\income.dev.txt"
    test_path = "data\\income-data\\income.test.txt"
    t=5
    itr=[1,2,3,4,5]
    c=[10**-4,10**-3,10**-2,10**-1,1]
    
    (train_x, train_y, dev_x, dev_y, test_x, test_y) = read_data(train_path, dev_path, test_path)
    
    w,mistakes_p,train_acc_p,dev_acc_p,test_acc_p = train_perc(train_x, train_y, dev_x, dev_y, test_x, test_y, t )
    w1,mistakes_pa,train_acc_pa,dev_acc_pa,test_acc_pa = train_passiveagg(train_x, train_y, dev_x, dev_y, test_x, test_y, t )
    
    
    graph(itr,mistakes_p,'Number of iterations','Number of mistakes','online learning curve for perceptron','perceptron_mistake.png')
    graph(itr,mistakes_p,'Number of iterations','Number of mistakes','online learning curve for passive aggressive','Passive_aggressive.png')
    
    
    graph(itr,train_acc_p,'Number of mistakes','Accuracy of Train data','Accuracy curve for training data in perceptron','Accuracy_perceptron.png')
    graph(itr,dev_acc_p,'Number of mistakes','Accuracy of development data','Accuracy curve for development data in perceptron','development_perceptron.png')
    graph(itr,test_acc_p,'Number of mistakes','Accuracy of Test data','Accuracy curve for test data in perceptron','Test_perceptron.png')
    graph(itr,train_acc_pa,'Number of mistakes','Accuracy of Train data','Accuracy curve for training data in passive aggressive','Passive_aggressive.png')
    graph(itr,dev_acc_pa,'Number of mistakes','Accuracy of development data','Accuracy curve for development data in passive aggressive','development_pa.png')
    graph(itr,test_acc_pa,'Number of mistakes','Accuracy of Test data','Accuracy curve for test data in passive aggressive','test_pa.png')
   
    
    w,b,acc1_a,acc2_a,acc3_a=avg_per(train_x, train_y, dev_x, dev_y, test_x, test_y,t)
    
    print('Training data accuracy of average perceptron(smart way) :', acc1_a)
    print('Development data accuracy of average perceptron(smart way) :',acc2_a)
    print('Test data accuracy of average perceptron(smart way) :',acc3_a)
    
    wsum,acc1,acc2,acc3=naive(train_x, train_y, dev_x, dev_y, test_x, test_y,t)
    print('Training data accuracy of average perceptron(naive way) :',acc1)
    print('Development data accuracy of average perceptron(naive way) :',acc2)
    print('Test data accuracy of average perceptron(naive way) :',acc3)
    
    i, dev_p, test_p=curve_p(train_x, train_y, dev_x, dev_y, test_x, test_y)
    i_pa, dev_pa, test_pa=curve_pa(train_x, train_y, dev_x, dev_y, test_x, test_y)
    
    graph(i,dev_p,'number of training examples','Accuracy of development data','general learning curve for perceptron development data ','gn-pda.png')
    graph(i,test_p,'number of training examples','Accuracy of test data','general learning curve for perceptron test data ','gen_glc.png')
    graph(i,dev_pa,'number of training examples','Accuracy of development data','general learning curve for passive aggressive development data ','gen_pad.png')
    graph(i,test_pa,'number of training examples','Accuracy of test data','general learning curve for passive aggressive test data ','gen_pat.png')
    
    train_svm(train_x, train_y, dev_x, dev_y, test_x, test_y)
    train_svmpoly(train_x, train_y, dev_x, dev_y, test_x, test_y)
    

time taken to perform 5 iterations of smart way average perceptron in sec 0.59375
Training data accuracy of average perceptron(smart way) : 0.73656
Development data accuracy of average perceptron(smart way) : 0.8249336870026525
Test data accuracy of average perceptron(smart way) : 0.8032634032634033
time taken to perform 5 iterations of naive way average perceptron in sec  0.484375
Training data accuracy of average perceptron(naive way) : 0.73544
Development data accuracy of average perceptron(naive way) : 0.8196286472148542
Test data accuracy of average perceptron(naive way) : 0.8046620046620047
0.7516 0.7606100795755968 0.7393939393939394
0.80708 0.8176392572944297 0.799067599067599
0.82744 0.8355437665782494 0.8195804195804196
0.82972 0.833554376657825 0.8228438228438228
0.83012 0.8309018567639257 0.8233100233100233
0.83476 0.8395225464190982 0.8331002331002331
0.83476 0.8395225464190982 0.8331002331002331
0.83984 0.8415119363395226 0.827972027972028
0.83984 0.8415119363395226 0.827